# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json as json

# Import API key
from api_key import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv("/Users/jasonlei/python-api-challenge/WeatherPy/output_data/cities.csv")
city_df.dropna()
city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,cidreira,97,BR,1619330012,88,-30.1811,-50.2056,68.00,10.25
1,okakarara,5,NaN,1619330013,36,-20.5833,17.4333,59.20,5.12
2,hithadhoo,14,MV,1619329689,67,-0.6000,73.0833,83.55,5.91
3,rikitea,100,PF,1619329690,77,-23.1203,-134.9692,78.42,11.72
4,esperance,26,AU,1619329690,26,-33.8667,121.9000,82.40,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"].astype(float)

In [4]:
# Heatmap of humidity
# Plot Heatmap
heat_map = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
heat_map.add_layer(heat_layer)
#Display figure
heat_map

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
temp_df = city_df.loc[(city_df['Max Temp'] >= 70)&(city_df['Max Temp'] <= 80)]
wind_speed_df = temp_df.loc[(temp_df['Wind Speed'] <= 10)]
weather_conditions = wind_speed_df.loc[(wind_speed_df['Cloudiness'] == 0)]
weather_conditions.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
145,hurghada,0,EG,1619329771,33,27.2574,33.8129,71.60,6.91
228,seoul,0,KR,1619329625,33,37.5683,126.9778,71.60,6.91
307,beloha,0,MG,1619329870,52,-25.1667,45.0500,79.65,8.23
425,murgab,0,TM,1619329942,43,37.4966,61.9714,71.60,4.61
517,caravelas,0,BR,1619329987,91,-17.7125,-39.2481,72.46,4.34


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = weather_conditions.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

In [7]:
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    long = row["Lng"]
    params["location"] = f"{lat},{long}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_name = requests.get(base_url, params=params)
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "next_page_token": "ATtYBwJDoANdvZJ-m0uf7oSWcPs8E-pkNRJsXOi7LlWu83JzhEy0wKyhQCRip7Pq4wutZwBRbWXcLu2FctrLlFL21yNq79zCtxux9LfpFnaTmw1mUl_Pfkj3wDKMn0fvxSIgm2N5Kp4XpeLb7Y_acc9H_Q9wD2tVTP-6VoFdcuEUzZyKLd-dG-ZlPM-n8SG-7X7d1u9UnXgbhEIuXjMC26d_2WkcE_4pWJzNKwXHe8KLgHcSFgJVTbdq3eOv9F9cZvaImla_P8VGShwISi5aUp8TYtTcpchaHEJz8ZoHQ-5EuUwHT7OFwwWiyZOkbd3OJT3X1-jp7ckmoNlyg4tVjqVZsD07FsY6H-sRh-yYbramGu44nt0ozDII4QVE24bVtGpLJ8hiFopkQXjkVQu3-A4bVCpSFyvQ-Zyy3Cq8E4FerNvBCUeQ_Z13Qve3riaZ4F6W",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 37.5653,
                    "lng": 126.980979
                },
                "viewport": {
                    "northeast": {
                        "lat": 37.56664982989273,
                        "lng": 126.982

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 37.5861441,
                    "lng": 61.8537656
                },
                "viewport": {
                    "northeast": {
                        "lat": 37.58758637989273,
                        "lng": 61.85517892989272
                    },
                    "southwest": {
                        "lat": 37.58488672010728,
                        "lng": 61.85247927010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Margush Hotel",
            "photos": [
                {
                    "height": 3265,
                    "html_attributions":

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,hurghada,0,EG,1619329771,33,27.2574,33.8129,71.60,6.91,
1,seoul,0,KR,1619329625,33,37.5683,126.9778,71.60,6.91,Lotte Hotel Seoul
2,beloha,0,MG,1619329870,52,-25.1667,45.0500,79.65,8.23,
3,murgab,0,TM,1619329942,43,37.4966,61.9714,71.60,4.61,Margush Hotel
4,caravelas,0,BR,1619329987,91,-17.7125,-39.2481,72.46,4.34,Cahy Praia Hotel


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

heat_map.add_layer(markers)
# Display figure
heat_map

Figure(layout=FigureLayout(height='420px'))